In [1]:
%matplotlib notebook
#add comments
import os
import sys
import csv
import pickle
from collections import Counter
from datetime import datetime

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch import nn
from torch.utils.data import DataLoader
from network import TweetyNet
import librosa
from librosa import display
from microfaune.audio import wav2spc, create_spec, load_wav
from glob import glob

from torch.utils.data import Dataset
from CustomAudioDataset import CustomAudioDataset

from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import scipy
import IPython.display as ipd

from TweetyNetModel import TweetyNetModel  

In [2]:
train = True
fineTuning = False
#needs at least 80 for mel spectrograms ## may be able to do a little less, but must be greater than 60
n_mels=72 # The closest we can get tmeporally is 72 with an output of 432 : i think it depends on whats good
#this number should be proportional to the length of the videos. 
datasets_dir = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing"
this_is_new_data = False

In [3]:
import pandas as pd
import numpy as np


#TODO Add mutliple species support for ROC curves 
def annotation_chunker_no_duplicates(kaleidoscope_df, chunk_length, include_no_bird=False, bird=None):
    """
    Function that converts a Kaleidoscope-formatted Dataframe containing 
    annotations to uniform chunks of chunk_length. If there
    are mutliple bird species in the same clip, this function creates chunks
    for the more confident bird species.

    Note: if all or part of an annotation covers the last < chunk_length
    seconds of a clip it will be ignored. If two annotations overlap in 
    the same 3 second chunk, both are represented in that chunk

    Args:
        kaleidoscope_df (Dataframe)
            - Dataframe of annotations in kaleidoscope format

        chunk_length (int)
            - duration to set all annotation chunks
    Returns:
        Dataframe of labels with chunk_length duration 
        (elements in "OFFSET" are divisible by chunk_length).
    """

    #Init list of clips to cycle through and output dataframe
    #kaleidoscope_df["FILEPATH"] =  kaleidoscope_df["FOLDER"] + kaleidoscope_df["IN FILE"] 
    kaleidoscope_df['FILEPATH'] = kaleidoscope_df.loc[:,['FOLDER','IN FILE']].sum(axis=1)
    clips = kaleidoscope_df["FILEPATH"].unique()
    df_columns = {'FOLDER': 'str', 'IN FILE' :'str', 'CLIP LENGTH' : 'float64', 'CHANNEL' : 'int64', 'OFFSET' : 'float64',
                'DURATION' : 'float64', 'SAMPLE RATE' : 'int64','MANUAL ID' : 'str'}
    output_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in df_columns.items()})
    
    # going through each clip
    for clip in clips:
        clip_df = kaleidoscope_df[kaleidoscope_df["FILEPATH"] == clip]
        path = clip_df["FOLDER"].unique()[0]
        file = clip_df["IN FILE"].unique()[0]
        birds = clip_df["MANUAL ID"].unique()
        sr = clip_df["SAMPLE RATE"].unique()[0]
        clip_len = clip_df["CLIP LENGTH"].unique()[0]

        # quick data sanitization to remove very short clips
        # do not consider any chunk that is less than chunk_length
        if clip_len < chunk_length:
            continue
        potential_annotation_count = int(clip_len)//int(chunk_length)

        # going through each species that was ID'ed in the clip
        arr_len = int(clip_len*1000)
        species_df = clip_df#[clip_df["MANUAL ID"] == bird]
        human_arr = np.zeros((arr_len))
        # looping through each annotation
        #print("========================================")
        for annotation in species_df.index:
            #print(species_df["OFFSET"][annotation])
            minval = int(round(species_df["OFFSET"][annotation] * 1000, 0))
            # Determining the end of a human label
            maxval = int(
                round(
                    (species_df["OFFSET"][annotation] +
                        species_df["DURATION"][annotation]) *
                    1000,
                    0))
            # Placing the label relative to the clip
            human_arr[minval:maxval] = 1
        # performing the chunk isolation technique on the human array

        for index in range(potential_annotation_count):
            #print("=======================")
            #print("-----------------------------------------")
            #print(index)
            chunk_start = index * (chunk_length*1000)
            chunk_end = min((index+1)*chunk_length*1000,arr_len)
            chunk = human_arr[int(chunk_start):int(chunk_end)]
            if max(chunk) >= 0.5:
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #Handle birdnet output edge case
                #print("-------------------------------------------")
                #print(sum(clip_df["DURATION"] == 3))
                #print(sum(clip_df["DURATION"] == 3)/clip_df.shape[0])
                #print("-------------------------------------------")
                if(sum(clip_df["DURATION"] == 3)/clip_df.shape[0] == 1):
                    #print("Processing here duration")
                    overlap = (clip_df["OFFSET"]+0.5 >= (annotation_start)) & (clip_df["OFFSET"]-0.5 <= (annotation_start))
                    annotation_df = clip_df[overlap]
                    #print(annotation_start, np.array(clip_df["OFFSET"]), overlap)
                    #print(annotation_df)
                else:
                    #print("Processing here")
                    overlap = is_overlap(clip_df["OFFSET"], clip_df["OFFSET"] + clip_df["DURATION"], annotation_start, annotation_start + chunk_length)
                    #print(overlap)
                    annotation_df = clip_df[overlap]
                    #print(annotation_df)
                
                #updating the dictionary
                if ('CONFIDENCE' in clip_df.columns):
                    annotation_df = annotation_df.sort_values(by="CONFIDENCE", ascending=False)
                    row["CONFIDENCE"] = annotation_df.iloc[0]["CONFIDENCE"]
                else:
                    #The case of manual id, or there is an annotation with no known confidence
                    row["CONFIDENCE"] = 1
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = annotation_df.iloc[0]["MANUAL ID"] 
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
            elif(include_no_bird):
                #print(max(chunk))
                #Get row data
                row = pd.DataFrame(index = [0])
                annotation_start = chunk_start / 1000

                #updating the dictionary
                row["CONFIDENCE"] = 0
                row["FOLDER"] = path
                row["IN FILE"] = file
                row["CLIP LENGTH"] = clip_len
                row["OFFSET"] = annotation_start
                row["DURATION"] = chunk_length
                row["SAMPLE RATE"] = sr
                row["MANUAL ID"] = "no bird"
                row["CHANNEL"] = 0
                output_df = pd.concat([output_df,row], ignore_index=True)
    
    return output_df


def is_overlap(offset_df, end_df, chunk_start, chunk_end):
    is_both_before = (chunk_end < offset_df) & (chunk_start < offset_df)
    is_both_after = (end_df < chunk_end) & (end_df < chunk_start)
    return (~is_both_before) & (~is_both_after)
    
    
    interval = pd.Interval(left=offset_df, right=end_df)
    print(interval)


def split_save_files(dataframe_of_data):
    chunked_df = annotation_chunker_no_duplicates(dataframe_of_data, 3, include_no_bird=True)
    
    
    chunked_df.apply()
    

In [26]:
from pydub import AudioSegment
import os

data = pd.read_csv("C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing/automated_cosmos_tweety_to_birdnet.csv")
chunked_df = annotation_chunker_no_duplicates(data, 3, include_no_bird=True)
chunked_df

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE
0,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,0.0,3.0,48000,Microcerculus marginatus,0.155038
1,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,3.0,3.0,48000,Centropus celebensis,0.020985
2,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,6.0,3.0,48000,Meliphaga notata,0.027436
3,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,9.0,3.0,48000,Microcerculus marginatus,0.331647
4,.\..\Cosmos_data\,XC100027 - Southern Nightingale-Wren - Microce...,81.528167,0,12.0,3.0,48000,Sciaphylax hemimelaena,0.017806
...,...,...,...,...,...,...,...,...,...
39712,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,0.0,3.0,48000,Thraupis episcopus,0.146423
39713,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,3.0,3.0,48000,Pyrrhocorax pyrrhocorax,0.042745
39714,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,6.0,3.0,48000,Thraupis sayaca,0.108350
39715,.\..\Cosmos_data\,XC9881 - Blue-grey Tanager - Thraupis episcopu...,14.700021,0,9.0,3.0,48000,no bird,0.000000


In [27]:
chunked_df.iloc[0]["IN FILE"]

'XC100027 - Southern Nightingale-Wren - Microcerculus marginatus.mp3'

In [28]:
import pydub
pydub.AudioSegment.ffmpeg = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"

In [29]:
from pydub import AudioSegment
AudioSegment.converter = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"
import os

path_to_audio = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing"
data_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split"
new_folder = data_path

from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path_to_audio) if isfile(join(path_to_audio, f))]
chunked_df = chunked_df[chunked_df["IN FILE"].isin(onlyfiles)]


chunked_renamed_df = chunked_df.copy()
count = 0
for filename in np.unique(chunked_df["IN FILE"]):
    file_df = chunked_df[chunked_df["IN FILE"] == filename]
    file, file_type = filename.split(".")
    for index,row in file_df.iterrows():
        t1 = row["OFFSET"] * 1000
        # end time in milliseconds
        t2 = (row["OFFSET"] + 3) * 1000
        new_name = file + "_" + str(int(row["OFFSET"])) +"." + file_type
        new_path = new_folder + "/" + new_name
        
        if(this_is_new_data):
            newAudio = AudioSegment.from_mp3(path_to_audio + "/" + filename)
            newAudio = newAudio[t1:t2]
            newAudio.export(new_path, format="mp3")

        chunked_renamed_df.at[index, "IN FILE"] = new_name
        chunked_renamed_df.at[index, "FOLDER"] = new_folder
        count += 1
        print(count, "out of 6451")

        
chunked_renamed_df = chunked_renamed_df[chunked_renamed_df["IN FILE"] != "TO DELETE"]
chunked_renamed_df
    
    

1 out of 6451
2 out of 6451
3 out of 6451
4 out of 6451
5 out of 6451
6 out of 6451
7 out of 6451
8 out of 6451
9 out of 6451
10 out of 6451
11 out of 6451
12 out of 6451
13 out of 6451
14 out of 6451
15 out of 6451
16 out of 6451
17 out of 6451
18 out of 6451
19 out of 6451
20 out of 6451
21 out of 6451
22 out of 6451
23 out of 6451
24 out of 6451
25 out of 6451
26 out of 6451
27 out of 6451
28 out of 6451
29 out of 6451
30 out of 6451
31 out of 6451
32 out of 6451
33 out of 6451
34 out of 6451
35 out of 6451
36 out of 6451
37 out of 6451
38 out of 6451
39 out of 6451
40 out of 6451
41 out of 6451
42 out of 6451
43 out of 6451
44 out of 6451
45 out of 6451
46 out of 6451
47 out of 6451
48 out of 6451
49 out of 6451
50 out of 6451
51 out of 6451
52 out of 6451
53 out of 6451
54 out of 6451
55 out of 6451
56 out of 6451
57 out of 6451
58 out of 6451
59 out of 6451
60 out of 6451
61 out of 6451
62 out of 6451
63 out of 6451
64 out of 6451
65 out of 6451
66 out of 6451
67 out of 6451
68 o

546 out of 6451
547 out of 6451
548 out of 6451
549 out of 6451
550 out of 6451
551 out of 6451
552 out of 6451
553 out of 6451
554 out of 6451
555 out of 6451
556 out of 6451
557 out of 6451
558 out of 6451
559 out of 6451
560 out of 6451
561 out of 6451
562 out of 6451
563 out of 6451
564 out of 6451
565 out of 6451
566 out of 6451
567 out of 6451
568 out of 6451
569 out of 6451
570 out of 6451
571 out of 6451
572 out of 6451
573 out of 6451
574 out of 6451
575 out of 6451
576 out of 6451
577 out of 6451
578 out of 6451
579 out of 6451
580 out of 6451
581 out of 6451
582 out of 6451
583 out of 6451
584 out of 6451
585 out of 6451
586 out of 6451
587 out of 6451
588 out of 6451
589 out of 6451
590 out of 6451
591 out of 6451
592 out of 6451
593 out of 6451
594 out of 6451
595 out of 6451
596 out of 6451
597 out of 6451
598 out of 6451
599 out of 6451
600 out of 6451
601 out of 6451
602 out of 6451
603 out of 6451
604 out of 6451
605 out of 6451
606 out of 6451
607 out of 6451
608 out 

1188 out of 6451
1189 out of 6451
1190 out of 6451
1191 out of 6451
1192 out of 6451
1193 out of 6451
1194 out of 6451
1195 out of 6451
1196 out of 6451
1197 out of 6451
1198 out of 6451
1199 out of 6451
1200 out of 6451
1201 out of 6451
1202 out of 6451
1203 out of 6451
1204 out of 6451
1205 out of 6451
1206 out of 6451
1207 out of 6451
1208 out of 6451
1209 out of 6451
1210 out of 6451
1211 out of 6451
1212 out of 6451
1213 out of 6451
1214 out of 6451
1215 out of 6451
1216 out of 6451
1217 out of 6451
1218 out of 6451
1219 out of 6451
1220 out of 6451
1221 out of 6451
1222 out of 6451
1223 out of 6451
1224 out of 6451
1225 out of 6451
1226 out of 6451
1227 out of 6451
1228 out of 6451
1229 out of 6451
1230 out of 6451
1231 out of 6451
1232 out of 6451
1233 out of 6451
1234 out of 6451
1235 out of 6451
1236 out of 6451
1237 out of 6451
1238 out of 6451
1239 out of 6451
1240 out of 6451
1241 out of 6451
1242 out of 6451
1243 out of 6451
1244 out of 6451
1245 out of 6451
1246 out of 64

1760 out of 6451
1761 out of 6451
1762 out of 6451
1763 out of 6451
1764 out of 6451
1765 out of 6451
1766 out of 6451
1767 out of 6451
1768 out of 6451
1769 out of 6451
1770 out of 6451
1771 out of 6451
1772 out of 6451
1773 out of 6451
1774 out of 6451
1775 out of 6451
1776 out of 6451
1777 out of 6451
1778 out of 6451
1779 out of 6451
1780 out of 6451
1781 out of 6451
1782 out of 6451
1783 out of 6451
1784 out of 6451
1785 out of 6451
1786 out of 6451
1787 out of 6451
1788 out of 6451
1789 out of 6451
1790 out of 6451
1791 out of 6451
1792 out of 6451
1793 out of 6451
1794 out of 6451
1795 out of 6451
1796 out of 6451
1797 out of 6451
1798 out of 6451
1799 out of 6451
1800 out of 6451
1801 out of 6451
1802 out of 6451
1803 out of 6451
1804 out of 6451
1805 out of 6451
1806 out of 6451
1807 out of 6451
1808 out of 6451
1809 out of 6451
1810 out of 6451
1811 out of 6451
1812 out of 6451
1813 out of 6451
1814 out of 6451
1815 out of 6451
1816 out of 6451
1817 out of 6451
1818 out of 64

2298 out of 6451
2299 out of 6451
2300 out of 6451
2301 out of 6451
2302 out of 6451
2303 out of 6451
2304 out of 6451
2305 out of 6451
2306 out of 6451
2307 out of 6451
2308 out of 6451
2309 out of 6451
2310 out of 6451
2311 out of 6451
2312 out of 6451
2313 out of 6451
2314 out of 6451
2315 out of 6451
2316 out of 6451
2317 out of 6451
2318 out of 6451
2319 out of 6451
2320 out of 6451
2321 out of 6451
2322 out of 6451
2323 out of 6451
2324 out of 6451
2325 out of 6451
2326 out of 6451
2327 out of 6451
2328 out of 6451
2329 out of 6451
2330 out of 6451
2331 out of 6451
2332 out of 6451
2333 out of 6451
2334 out of 6451
2335 out of 6451
2336 out of 6451
2337 out of 6451
2338 out of 6451
2339 out of 6451
2340 out of 6451
2341 out of 6451
2342 out of 6451
2343 out of 6451
2344 out of 6451
2345 out of 6451
2346 out of 6451
2347 out of 6451
2348 out of 6451
2349 out of 6451
2350 out of 6451
2351 out of 6451
2352 out of 6451
2353 out of 6451
2354 out of 6451
2355 out of 6451
2356 out of 64

...

3375 out of 6451
3376 out of 6451
3377 out of 6451
3378 out of 6451
3379 out of 6451
3380 out of 6451
3381 out of 6451
3382 out of 6451
3383 out of 6451
3384 out of 6451
3385 out of 6451
3386 out of 6451
3387 out of 6451
3388 out of 6451
3389 out of 6451
3390 out of 6451
3391 out of 6451
3392 out of 6451
3393 out of 6451
3394 out of 6451
3395 out of 6451
3396 out of 6451
3397 out of 6451
3398 out of 6451
3399 out of 6451
3400 out of 6451
3401 out of 6451
3402 out of 6451
3403 out of 6451
3404 out of 6451
3405 out of 6451
3406 out of 6451
3407 out of 6451
3408 out of 6451
3409 out of 6451
3410 out of 6451
3411 out of 6451
3412 out of 6451
3413 out of 6451
3414 out of 6451
3415 out of 6451
3416 out of 6451
3417 out of 6451
3418 out of 6451
3419 out of 6451
3420 out of 6451
3421 out of 6451
3422 out of 6451
3423 out of 6451
3424 out of 6451
3425 out of 6451
3426 out of 6451
3427 out of 6451
3428 out of 6451
3429 out of 6451
3430 out of 6451
3431 out of 6451
3432 out of 6451
3433 out of 64

3925 out of 6451
3926 out of 6451
3927 out of 6451
3928 out of 6451
3929 out of 6451
3930 out of 6451
3931 out of 6451
3932 out of 6451
3933 out of 6451
3934 out of 6451
3935 out of 6451
3936 out of 6451
3937 out of 6451
3938 out of 6451
3939 out of 6451
3940 out of 6451
3941 out of 6451
3942 out of 6451
3943 out of 6451
3944 out of 6451
3945 out of 6451
3946 out of 6451
3947 out of 6451
3948 out of 6451
3949 out of 6451
3950 out of 6451
3951 out of 6451
3952 out of 6451
3953 out of 6451
3954 out of 6451
3955 out of 6451
3956 out of 6451
3957 out of 6451
3958 out of 6451
3959 out of 6451
3960 out of 6451
3961 out of 6451
3962 out of 6451
3963 out of 6451
3964 out of 6451
3965 out of 6451
3966 out of 6451
3967 out of 6451
3968 out of 6451
3969 out of 6451
3970 out of 6451
3971 out of 6451
3972 out of 6451
3973 out of 6451
3974 out of 6451
3975 out of 6451
3976 out of 6451
3977 out of 6451
3978 out of 6451
3979 out of 6451
3980 out of 6451
3981 out of 6451
3982 out of 6451
3983 out of 64

4424 out of 6451
4425 out of 6451
4426 out of 6451
4427 out of 6451
4428 out of 6451
4429 out of 6451
4430 out of 6451
4431 out of 6451
4432 out of 6451
4433 out of 6451
4434 out of 6451
4435 out of 6451
4436 out of 6451
4437 out of 6451
4438 out of 6451
4439 out of 6451
4440 out of 6451
4441 out of 6451
4442 out of 6451
4443 out of 6451
4444 out of 6451
4445 out of 6451
4446 out of 6451
4447 out of 6451
4448 out of 6451
4449 out of 6451
4450 out of 6451
4451 out of 6451
4452 out of 6451
4453 out of 6451
4454 out of 6451
4455 out of 6451
4456 out of 6451
4457 out of 6451
4458 out of 6451
4459 out of 6451
4460 out of 6451
4461 out of 6451
4462 out of 6451
4463 out of 6451
4464 out of 6451
4465 out of 6451
4466 out of 6451
4467 out of 6451
4468 out of 6451
4469 out of 6451
4470 out of 6451
4471 out of 6451
4472 out of 6451
4473 out of 6451
4474 out of 6451
4475 out of 6451
4476 out of 6451
4477 out of 6451
4478 out of 6451
4479 out of 6451
4480 out of 6451
4481 out of 6451
4482 out of 64

4957 out of 6451
4958 out of 6451
4959 out of 6451
4960 out of 6451
4961 out of 6451
4962 out of 6451
4963 out of 6451
4964 out of 6451
4965 out of 6451
4966 out of 6451
4967 out of 6451
4968 out of 6451
4969 out of 6451
4970 out of 6451
4971 out of 6451
4972 out of 6451
4973 out of 6451
4974 out of 6451
4975 out of 6451
4976 out of 6451
4977 out of 6451
4978 out of 6451
4979 out of 6451
4980 out of 6451
4981 out of 6451
4982 out of 6451
4983 out of 6451
4984 out of 6451
4985 out of 6451
4986 out of 6451
4987 out of 6451
4988 out of 6451
4989 out of 6451
4990 out of 6451
4991 out of 6451
4992 out of 6451
4993 out of 6451
4994 out of 6451
4995 out of 6451
4996 out of 6451
4997 out of 6451
4998 out of 6451
4999 out of 6451
5000 out of 6451
5001 out of 6451
5002 out of 6451
5003 out of 6451
5004 out of 6451
5005 out of 6451
5006 out of 6451
5007 out of 6451
5008 out of 6451
5009 out of 6451
5010 out of 6451
5011 out of 6451
5012 out of 6451
5013 out of 6451
5014 out of 6451
5015 out of 64

ValueError: too many values to unpack (expected 2)

In [30]:
x = chunked_renamed_df.groupby(["IN FILE", "OFFSET"]).count().sort_values(by="MANUAL ID", ascending=False)
x.shape[0]

6449

In [31]:
def show_mel_spectrogram(spec, uid):
    fig, ax = plt.subplots()
    M_db_bird = librosa.power_to_db(spec, ref=np.max)
    img = librosa.display.specshow(M_db_bird, y_axis='mel', x_axis='time', ax=ax)
    ax.set(title=uid)
    plt.show()

In [32]:
def load_dataset(data_path, use_dump=True, csv="labels.csv", kalediscope=False):
    mel_dump_file = os.path.join(data_path, "mel_dataset.pkl")
    if os.path.exists(mel_dump_file) and use_dump:
        with open(mel_dump_file, "rb") as f:
            dataset = pickle.load(f)
    
    elif kalediscope:
        dataset = compute_feature_kaledoscope_labels(data_path, csv)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    else:
        dataset = compute_feature(data_path, csv)
        with open(mel_dump_file, "wb") as f:
            pickle.dump(dataset, f)
    inds = [i for i, x in enumerate(dataset["X"]) if x.shape[1] == 431]
    X = np.array([dataset["X"][i].transpose() for i in inds])
    Y = np.array([int(dataset["Y"][i]) for i in inds])
    uids = [dataset["uids"][i] for i in inds]
    return X, Y, uids

def compute_feature(data_path, csv = "labels.csv"):
    print(f"Compute features for dataset {os.path.basename(data_path)}")
    labels_file = os.path.join(data_path, csv)
    print(labels_file)
    if os.path.exists(labels_file):
        with open(labels_file, "r") as f:
            reader = csv.reader(f, delimiter=',')
            labels = {}
            next(reader)  # pass fields names
            for name, _, y in reader:
                labels[name] = y
    else:
        print("Warning: no label file detected.")
        wav_files = glob(os.path.join(data_path, "wav/*.wav"))
        labels = {os.path.basename(f)[:-4]: None for f in wav_files}
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        X.append(spc)
        Y.append(y)
        uids.append(file_id)
        i += 1
    return {"uids": uids, "X": X, "Y": Y}

def compute_feature_df(data_path, df):
    
    i = 1
    X = []
    Y = []
    uids = []
    for file_id, y in labels.items():
        print(f"{i:04d}/{len(labels)}: {file_id:20s}", end="\r")
        spc = wav2spc(os.path.join(data_path, "wav", f"{file_id}.wav"), n_mels=n_mels)
        X.append(spc)
        Y.append(y)
        uids.append(file_id)
        i += 1
    return {"uids": uids, "X": X, "Y": Y}

def split_dataset(X, Y, test_size=0.2, random_state=0):
    split_generator = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    ind_train, ind_test = next(split_generator.split(X, Y))
    X_train, X_test = X[ind_train, :, :], X[ind_test, :, :]
    Y_train, Y_test = Y[ind_train], Y[ind_test]
    return ind_train, ind_test

In [33]:
species = list(np.unique(chunked_renamed_df["MANUAL ID"]))
chunked_renamed_df["y"] = chunked_renamed_df["MANUAL ID"].apply(lambda x: species.index(x))
chunked_renamed_df

,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE,y
201,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,XC100864 - Rufous-collared Sparrow - Zonotrich...,9.299604,0,0.0,3.0,48000,Zonotrichia capensis,0.981316,959
202,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,XC100864 - Rufous-collared Sparrow - Zonotrich...,9.299604,0,3.0,3.0,48000,Zonotrichia capensis,0.455129,959
203,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,XC100864 - Rufous-collared Sparrow - Zonotrich...,9.299604,0,6.0,3.0,48000,Zonotrichia capensis,0.958481,959
268,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,XC101469 - Pale-breasted Thrush - Turdus leuco...,137.377979,0,0.0,3.0,48000,Mimus gilvus,0.130503,491
269,C:/Users/Siloux/Desktop/E4E/passive-acoustic-b...,XC101469 - Pale-breasted Thrush - Turdus leuco...,137.377979,0,3.0,3.0,48000,Mimus gilvus,0.066429,491
...,...,...,...,...,...,...,...,...,...,...
39645,.\..\Cosmos_data\,XC97532 - Blue-grey Tanager - Thraupis episcop...,20.297146,0,15.0,3.0,48000,Bubo ascalaphus,0.013156,82
39646,.\..\Cosmos_data\,XC97542 - Slate-throated Whitestart - Myioboru...,4.362458,0,0.0,3.0,48000,Thlypopsis sordida,0.023451,861
39647,.\..\Cosmos_data\,XC9766 - Rufous-collared Sparrow - Zonotrichia...,11.232667,0,0.0,3.0,48000,Empidonax traillii,0.121914,302
39648,.\..\Cosmos_data\,XC9766 - Rufous-collared Sparrow - Zonotrichia...,11.232667,0,3.0,3.0,48000,Merops pusillus,0.040642,480


In [34]:
import os
from pydub import AudioSegment

# You dont need the number of files in the folder, just iterate over them directly using:
if (this_is_new_data):
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(folder + '/' + file)
        if ext == ".mp3":
            #os.remove(folder + '/' + file) 

            mp3_sound = AudioSegment.from_mp3(folder + '/' + file)
            mp3_sound.export("{0}.wav".format(name), format="wav")

In [35]:
import os
from pydub import AudioSegment

if(this_is_new_data):
    # You dont need the number of files in the folder, just iterate over them directly using:
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        #spliting the file into the name and the extension
        name, ext = os.path.splitext(file)
        if ext == ".mp3":
            print(folder)
            try:
                os.remove(folder + "/" + file) 
            except Exception as e:
                print(e)
            

In [36]:
chunked_renamed_df["IN FILE"][201]

'XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_0.mp3'

In [37]:
chunked_renamed_df["IN FILE"] = chunked_renamed_df["IN FILE"].apply(lambda x: x.replace(".mp3", ".wav"))

In [38]:
import os
from pydub import AudioSegment

# You dont need the number of files in the folder, just iterate over them directly using:
if(this_is_new_data):
    for index, row in chunked_renamed_df.iterrows():
        file = row["IN FILE"]
        folder = row["FOLDER"]
        sound = AudioSegment.from_wav(folder + "/" + file)
        sound = sound.set_channels(1)
        sound.export(folder + "/" + file, format="wav")


In [39]:
data_path = "C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split"
i = 1
X = []
Y = []
uids = []
for index, row in chunked_renamed_df.iterrows():
    try:
        print(f"{i:04d}/{chunked_renamed_df.shape[0]}", end="\r")
        print(os.path.join(data_path, row["IN FILE"]))
        spc = wav2spc(os.path.join(data_path, row["IN FILE"]), n_mels=n_mels)
        X.append(spc)
        Y.append(row["y"])
        uids.append(row["IN FILE"])
        i += 1
    except:
        continue
dataset = {"uids": uids, "X": X, "Y": Y}

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_0.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_3.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_6.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101469 - Pale-breasted Thrush - Turdus leucomelas_0.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101469 - Pale-breasted Thrush - Turdus leucomelas_3.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/Tw

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101564 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_9.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101564 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_12.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101564 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_15.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101564 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_18.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC101564 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_21.wav
C:/Users/Siloux/

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC103462 - Green-backed Trogon - Trogon viridis_0.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC103462 - Green-backed Trogon - Trogon viridis_3.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC103462 - Green-backed Trogon - Trogon viridis_6.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC103462 - Green-backed Trogon - Trogon viridis_9.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC103462 - Green-backed Trogon - Trogon viridis_12.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC103462 - Green-back

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC104667 - Rufous-collared Sparrow - Zonotrichia capensis_54.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC104667 - Rufous-collared Sparrow - Zonotrichia capensis_57.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC104667 - Rufous-collared Sparrow - Zonotrichia capensis_60.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC104667 - Rufous-collared Sparrow - Zonotrichia capensis_63.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC104667 - Rufous-collared Sparrow - Zonotrichia capensis_66.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmo

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC117365 - White-throated Toucan - Ramphastos tucanus cuvieri_0.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC117365 - White-throated Toucan - Ramphastos tucanus cuvieri_3.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC117365 - White-throated Toucan - Ramphastos tucanus cuvieri_6.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC117365 - White-throated Toucan - Ramphastos tucanus cuvieri_9.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC117365 - White-throated Toucan - Ramphastos tucanus cuvieri_12.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/c

...

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC63638 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC63638 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC63638 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC63638 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC63638 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC693388 - Scaly-breasted Woodpecker - Celeus grammicus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC693388 - Scaly-breasted Woodpecker - Celeus grammicus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC693388 - Scaly-breasted Woodpecker - Celeus grammicus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC693389 - Scaly-breasted Woodpecker - Celeus grammicus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC693389 - Scaly-breasted Woodpecker - Celeus grammicus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processin

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC708459 - Pale-breasted Thrush - Turdus leucomelas leucomelas.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC708459 - Pale-breasted Thrush - Turdus leucomelas leucomelas.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC708459 - Pale-breasted Thrush - Turdus leucomelas leucomelas.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC708459 - Pale-breasted Thrush - Turdus leucomelas leucomelas.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC708459 - Pale-breasted Thrush - Turdus leucomelas leucomelas.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC73831 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC73831 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC73831 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC73831 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC73831 - Southern Nightingale-Wren - Microcerculus marginatus.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_

C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC91002 - Rufous-collared Sparrow - Zonotrichia capensis.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC91002 - Rufous-collared Sparrow - Zonotrichia capensis.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC91002 - Rufous-collared Sparrow - Zonotrichia capensis.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC91002 - Rufous-collared Sparrow - Zonotrichia capensis.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_processing_split\XC91002 - Rufous-collared Sparrow - Zonotrichia capensis.wav
C:/Users/Siloux/Desktop/E4E/passive-acoustic-biodiversity/TweetyNET/cosmos_data/cosmos_random_sample_proc

In [40]:
dataset

{'uids': ['XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_0.wav',
  'XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_3.wav',
  'XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis_6.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_0.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_3.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_6.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_9.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_12.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_15.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_18.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_21.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_24.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_27.wav',
  'XC101469 - Pale-breasted Thrush - Turdus leucomelas_30.wav',
  'XC101469 - Pale-breasted Thrush - Turdus

In [41]:


inds = [i for i, x in enumerate(dataset["X"]) if x.shape[1] == 130]
X = np.array([dataset["X"][i].transpose() for i in inds])
Y = np.array([int(dataset["Y"][i]) for i in inds])
uids = [dataset["uids"][i] for i in inds]
X, Y, uids


(array([[[24.874008, 26.37475 , 24.874008, ..., 24.874008, 24.874008,
          24.874008],
         [63.904823, 61.36703 , 62.28552 , ..., 24.874008, 24.874008,
          24.874008],
         [70.09499 , 71.15409 , 72.341385, ..., 24.874008, 24.874008,
          24.874008],
         ...,
         [69.60951 , 68.95202 , 66.616356, ..., 24.874008, 24.874008,
          24.874008],
         [69.00752 , 68.65988 , 71.341705, ..., 24.874008, 24.874008,
          24.874008],
         [63.309593, 69.6176  , 69.48613 , ..., 24.874008, 24.874008,
          24.874008]],
 
        [[55.952545, 59.178352, 65.13502 , ..., 19.280636, 18.998732,
          18.515867],
         [62.761135, 69.90245 , 70.82655 , ..., 10.783119, 10.783119,
          10.783119],
         [68.85659 , 64.67638 , 66.7065  , ..., 10.783119, 10.783119,
          10.783119],
         ...,
         [69.900375, 67.10619 , 70.212036, ..., 10.783119, 10.783119,
          10.783119],
         [64.21452 , 67.490616, 65.32551 , ..., 1

In [42]:
dataset["X"][0].shape[1]


130

In [ ]:
dataset

In [ ]:
#X0, Y0, uids0 = load_dataset(datasets_dir, csv="automated_cosmos_tweety_to_file.csv")#os.path.join(datasets_dir, "ff1010bird_wav"))
#print(X0)
#uids0, X0, Y0 =  uids, X,Y#load_dataset(os.path.join(datasets_dir, "warblrb10k_public_wav"))

#print(X0.shape)
#print(Y0.shape)
#print(len(uids0))

#X = np.concatenate([X0]).astype(np.float32)/255
#Y = np.concatenate([Y0])
#uids = np.concatenate([uids0])
#print(X.shape, Y.shape, uids.shape)
#del X0, Y0, uids0, X1, Y1, uids1 

In [ ]:
Counter(Y)

Actual work

In [ ]:
print(type(uids))
uids = np.array(uids)

In [ ]:
ind_train_val, ind_test = split_dataset(X, Y)
ind_train, ind_val = split_dataset(X[ind_train_val, :, :, np.newaxis], Y[ind_train_val], test_size=0.1)
X_train, X_test, X_val = X[ind_train, :, :, np.newaxis], X[ind_test, :, :, np.newaxis], X[ind_val, :, :, np.newaxis]
Y_train, Y_test, Y_val = Y[ind_train], Y[ind_test], Y[ind_val]
uids_train, uids_test, uids_val = uids[ind_train], uids[ind_test], uids[ind_val]
#del X, Y

print("Training set: ", Counter(Y_train))
print("Test set: ", Counter(Y_test))
print("Validation set: ", Counter(Y_val))
#very imbalanced dataset

In [ ]:
len(Counter(Y_train))
X_train[0].shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
tweetynet = TweetyNetModel(len(Counter(Y_train)), (1, n_mels, 431), device, binary=True)

In [ ]:
type(train_dataset[:10])

In [ ]:
train_dataset = train_dataset.type(torch.LongTensor)
test_dataset = test_dataset.type(torch.LongTensor)
val_dataset = val_dataset.type(torch.LongTensor)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload




history, test_out, start_time, end_time = tweetynet.train_pipeline(train_dataset, val_dataset, test_dataset, 
                                                                   lr=.001, batch_size=6,epochs=5, save_me=True,
                                                                   fine_tuning=False, finetune_path=None)

In [ ]:
end_time - start_time

In [ ]:
test_out


In [ ]:
chunked_renamed_df.to_csv("formated_tweetynet_ground_truth.csv")

# Visualization

In [ ]:
#should categorize this data

In [ ]:
a, sr = librosa.load(os.path.join(datasets_dir, folder_name, "wav", uid+".wav"))
sr

In [ ]:
#-100,-2
idx_here = -100
x = X[idx_here]
y = Y[idx_here]
uid = uids[idx_here]
folder_name = "ff1010bird_wav" if "-" not in uid else "warblrb10k_public_wav"
sr = 44100
x.shape

In [ ]:
show_mel_spectrogram(x.transpose(), uid)
y, uid

In [ ]:
ipd.Audio(os.path.join(datasets_dir, folder_name, "wav", uid+".wav"))

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(a, sr=sr)